In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Text vectorization :
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Different models to try :
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# Data handling :
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics

import pickle
from scipy.sparse import hstack

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

## Import des mails dans un dataframe :

In [3]:
df = pd.read_excel('DATA.xlsx')
df.rename(columns={'created':'Date',
                   'adresse_mail_expediteur':'Expéditeur',
                   'nom_projet':'Projet',
                   'doc_mail_destinataire':'Destinataire'},
         inplace=True)
df = df[['Objet','Expéditeur','Destinataire','Projet']]
df.head()

,Objet,Expéditeur,Destinataire,Projet
0,"Merci d avoir modifié TEAMBER, Logiciel gestio...",noreply-maps-issues@google.com,alexis.cavaroc@teamber.fr,AGENCE DE COMMUNICATION
1,"Vielen Dank, dass Sie Änderungen für TEAMBER, ...",noreply-maps-issues@google.com,alexis.cavaroc@teamber.fr,AGENCE DE COMMUNICATION
2,[TEAMBER] Notes réunion 08 Janvier,b.croquin@labsoft.fr,l.bancel@labsoft.fr,DEVELOPPEMENT
3,[TEAMBER] Notes réunion 08 Janvier,b.croquin@labsoft.fr,teamber@teamber.fr,DEVELOPPEMENT
4,[TEAMBER] Notes réunion 08 Janvier,b.croquin@labsoft.fr,teamber@labsoft.fr,DEVELOPPEMENT


## Modèle 1 : en utilisant simplement l'objet du mail 

Split du dataset en train/test 80/20

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['Objet'], df['Projet'], test_size=.2,shuffle=True)

In [12]:
text_vect = TfidfVectorizer()
X_train_vect = text_vect.fit_transform(X_train)
X_test_vect = text_vect.transform(X_test)
X_test_vect

<33969x7153 sparse matrix of type '<class 'numpy.float64'>'
	with 160227 stored elements in Compressed Sparse Row format>

Test de différents modèles :
- RandomForestClassifier
- LinearSVC
- MultinomialNB

In [15]:
models = [RandomForestClassifier(n_estimators=500, max_depth=5, random_state=0),
          LinearSVC(),
          MultinomialNB()]

print("#mails pour training :",X_train_vect.shape[0],'\n#projets :',y_train.factorize()[0].max())
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_train_vect,y_train)
    print('Accuracy',model_name,':',model.score(X_test_vect,y_test))

#mails pour training : 135876 
#projets : 132
Accuracy RandomForestClassifier : 0.23291824899172775
Accuracy LinearSVC : 0.7748829815419942
Accuracy MultinomialNB : 0.6139126850952339


Le meilleur modèle est le LinearSVC, on continue donc avec celui-là.

In [16]:
model = LinearSVC()
model.fit(X_train_tfidf,y_train)
y_pred = model.predict(X_test_tfidf)
print(metrics.classification_report(y_test, y_pred, target_names=y_test.unique()))

C:\Users\antoi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                              precision    recall  f1-score   support

               DEVELOPPEMENT       1.00      0.33      0.50         6
       LETELLIER ARCHITECTES       0.95      0.69      0.80       176
                   PLENETUDE       0.86      0.68      0.76       191
     AGENCE DE COMMUNICATION       1.00      1.00      1.00         2
                  OTCE INFRA       1.00      0.33      0.50         3
           TRACE ARCHITECTES       0.86      0.77      0.81       441
        FRANCOIS DE LA SERRE       0.89      0.81      0.85       262
                 ATELIER BLM       0.00      0.00      0.00         1
           GARCIA INGENIERIE       1.00      0.67      0.80         3
                       CETEC       0.72      0.55      0.62        97
                     LABSOFT       0.76      0.78      0.77       871
  MONACO INGENIERIE PARTNERS       1.00      0.25      0.40         4
              CABINET ECTARE       0.87      0.79      0.83        78
                 OT

In [16]:
df.dropna(inplace=True)

## Modèle 2 : on utilise l'objet, l'expéditeur et le destinataire.

In [34]:
cols_train = ['Objet','Expéditeur','Destinataire']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['Projet'].values)
y = le.transform(df['Projet'])

X_train, X_test, y_train, y_test = train_test_split(df[cols_train], y,
                                                    test_size=.2, shuffle = True)
X_train.head(2)

,Objet,Expéditeur,Destinataire
100214,RE: pb synchro agenda téléphone,alain.fourets@plenetude.fr,alexis.cavaroc@teamber.fr
43652,RE: [SPAM] TEAMBER - AG reconstitution des ca...,Christophe.BOLZON@banque-france.fr,sebastien.perrot@teamber.fr


In [35]:
def fit_vectorize(X, col):
    tfidf_vectorizer = TfidfVectorizer()
    X_tfidf = tfidf_vectorizer.fit_transform(X)
    pickle.dump(tfidf_vectorizer,       open("TfidfVectorizer" + col,'wb'))
    return X_tfidf

def vectorize(X,col):
    tfidf_vectorizer = pickle.load(open("TfidfVectorizer"+col,'rb'))
    X_tfidf = tfidf_vectorizer.transform(X)
    return X_tfidf

def fit_transform_textcolumns(X, textcolumns):
    col = textcolumns[0]
    out = fit_vectorize(X[col],col)
    for i in range(1,len(textcolumns)) :
        col = textcolumns[i]
        out_next = fit_vectorize(X[col],col)
        out = hstack((out, out_next))
    return out

def transform_textcolumns(X, textcolumns):
    col = textcolumns[0]
    out = vectorize(X[col],col)
    for i in range(1,len(textcolumns)) :
        col = textcolumns[i]
        out_next = vectorize(X[col],col)
        out = hstack((out, out_next))
    return out

In [36]:
textcolumns = ["Objet",'Expéditeur','Destinataire']
X_train_vect= fit_transform_textcolumns(X_train,textcolumns)
X_test_vect = transform_textcolumns(X_test,textcolumns)

##### Création et entraînement du modèle :

In [52]:
model = LinearSVC()
model.fit(X_train_vect,y_train)
print("Modèle entrainé sur : ",X_train_vect.shape[0],
      "mails\nPouvant être classés dans ",y_test.max(),
      'projets différents\nAccuracy : ',model.score(X_test_vect,y_test))

Modèle entrainé sur :  135806 mails
Pouvant être classés dans  133 projets différents
Accuracy :  0.9151154571159283


In [41]:
models = [RandomForestClassifier(n_estimators=500, max_depth=5, random_state=0),
          LinearSVC(),
          MultinomialNB()]

print("#mails pour training :",X_train_vect.shape[0],'\n#projets :',y_train.max())
for model in models:
    model_name = model.__class__.__name__
    model.fit(X_train_vect,y_train)
    print('Accuracy',model_name,':',model.score(X_test_vect,y_test))

#mails pour training : 135806 
#projets : 133
Accuracy RandomForestClassifier : 0.3800954288407163
Accuracy LinearSVC : 0.9151154571159283
Accuracy MultinomialNB : 0.8030749293119699


### On peut à proposer un top 3 grâce à un autre modèle : 

In [320]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.9210650329877474

In [321]:
def top3(model,le, x):
    tops = []
    for i in range(x.shape[0]):
        var = x.getrow(i)
        pp = clf.predict_proba(var)[0]
        if pp.max() < 1:
            index  = np.where(pp>0)[0] 
            scores = pp[index]
            ens = np.hstack([np.array([index]).T,np.array([scores]).T])
            preds = np.flip(ens[ens[:,1].argsort()],axis=0)
            top = preds[:,0].astype(int)[:min(3,len(preds))]
            top = list(le.inverse_transform(top))
        else :
            top = [le.inverse_transform(clf.predict(var))]
        tops.append(top)
    return tops

In [318]:
c = 0
y_test2 = le.inverse_transform(y_test)
for i in tqdm(range(X_test.shape[0])):
    pred = top3(clf,le,X_test.getrow(i))[0]
    if y_test2[i] in pred:
        c+=1
c/X_test.shape[0]

100%|███████████████████████████████████████████████████████████████████████████| 16976/16976 [01:33<00:00, 181.94it/s]


0.929370876531574

Conclusion : on ne gagne que 1% de précision, cela ne vaut donc pas le coup de faire une interface pour un top 3.

In [15]:
y_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

                              precision    recall  f1-score   support

            AA GROUP VALENCE       1.00      1.00      1.00         4
                     ADDENDA       0.95      0.93      0.94       152
     AGENCE DE COMMUNICATION       0.90      0.89      0.90       193
             AGENCE EMPEREUR       1.00      1.00      1.00         2
          AIRBUS HELICOPTERS       1.00      1.00      1.00         2
                      AKADOM       0.90      0.90      0.90       395
                     ALLIAGE       0.95      0.91      0.93       235
              ALU ARCHITECTE       0.00      0.00      0.00         1
                     ARCHING       1.00      0.25      0.40         4
                   ASTONWOOD       0.90      0.94      0.92        88
                 ATELIER BLM       0.93      0.91      0.92       963
               ATELIER LOYER       1.00      1.00      1.00         1
                        ATYC       1.00      0.95      0.97        75
                   

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [40]:
model.predict(X_test)

array(['GARCIA INGENIERIE', 'CABINET ECTARE', 'LETELLIER ARCHITECTES',
       ..., 'ATELIER BLM', 'PLENETUDE', 'GARCIA INGENIERIE'], dtype=object)

In [19]:
model.predict_proba(X_test)

AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

##### Sauvegarde du modèle :

In [41]:
pickle.dump(model,open("MailClassifier",'wb'))

## Modèle 3 : avec contenu du mail, date, envoyé/reçu 

In [17]:
cols_train = ['Objet','Expéditeur','Destinataire','Date']
X_train, X_test, y_train, y_test = train_test_split(df[cols_train], df['Projet'], test_size=.2, shuffle = True)
X_train.head(2)

,Objet,Expéditeur,Destinataire,Date
57191,RE: Chargé d affaires,ebarbier@sercq.fr,virginie.debede@teamber.fr,1585063432
119445,RE: Relation Teamber - Serveur local,alexis.cavaroc@teamber.fr,support@teamber.fr,1554297509


In [18]:
text_cols = ['Objet','Expéditeur','Destinataire']

X_train_text = fit_transform_textcolumns(X_train,text_cols)
X_test_text = transform_textcolumns(X_test,text_cols)

In [100]:
def to_sparse(X, cols_to_convert):
    from scipy.sparse import coo_matrix
    col = cols_to_convert[0]
    out = coo_matrix(X[col].values).transpose()
    for i in range(1,len(cols_to_convert)):
        out_next = coo_matrix(X[col].values).transpose()
        out = hstack((out,out_next))
    return out

In [46]:
X_train_date = to_sparse(X_train,['Date'])
X_test_date = to_sparse(X_test,['Date'])

In [47]:
xtrain = hstack((X_train_text,X_train_date))
xtest  = hstack((X_test_text, X_test_date))

##### Création et entraînement du modèle :

In [49]:
from tqdm import tqdm

In [51]:
batch_size = 64
model = LinearSVC()
DATASET_SIZE = xtrain.shape[0]
print(DATASET_SIZE)

135806


In [ ]:
model.fit(xtrain,y_train)

##### Test du modèle :

In [226]:
print("Modèle entrainé sur : ",xtrain.shape[0],"mails\nPouvant être classés dans ",y_train.max(),\
      'projets différents\nTaux de succès : ',model.score(xtest,y_test))

Modèle entrainé sur :  135806 mails
Pouvant être classés dans  132 projets différents
Taux de succès :  0.914997643732328


##### Sauvegarde du modèle :

In [227]:
from datetime import datetime
pickle.dump(model,open("MailClassifier"+datetime.today().strftime('%Y-%m-%d-%Hh%M'),'wb'))